# Intro to Q\# #
---

# Qubits
What are they and why do I want one?

From a mathematical perspective, a qubit can be represented by a vector of 2 complex numbers like this:

$\vec{x} = \left[\begin{matrix} 0.25 + 0\times i \\0.75 + 0\times i \end{matrix}\right]$

We will often use short hand to label these vectors with a special notation called a ket:

$\left|x\right>$

How do I get a qubit in Q#? 

Allocate one with a `using` 

```
using (qubit = Qubit[1]) {
       // We put here what we want to do with the qubit
}

// After this using block, the qubit(s) we used are returned back to the target machine
```

In [19]:
open Microsoft.Quantum.Extensions.Diagnostics;

operation PrepareQubit() : Unit {
    using (qubit = Qubit[1]) {     // We want a tuple of 1 qubit to use for our task
            DumpMachine();         // Print out what the simulator is keeping a record of
            ResetAll(qubit);       // We have to return our qubit to the simulator the same way we got it
    }
}

PrepareQubit

Cool! now we have an `operation` which is the main way to use qubits in Q#. We can run our operation `PrepareQubit` by asking the IQ# kernel to simulate that operation:

In [20]:
%simulate PrepareQubit

# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0


()

You can read the above output like the vector we wrote above, where the first column is the index, the second is the real part of the vector at that position, and the second is the complex part of that vector entry.

So the printout
```
0:	1	0
1:	0	0
```
is the same as 
$\left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

Since we are running on a simulator, we can also use some built-in utilities in the prelude to learn more about the qubit resources available:`

In [21]:
%simulate GetQubitsAvailableToUse 

32

## Quantum random number generator

In [27]:
operation Qrng() : Result {
    using (qubit = Qubit()) {

        H(qubit);
       
        return MResetZ(qubit);
    }
}

C:/snippet:30b12e50-b7e8-41a9-aaad-a933ace9be5f(2,16): error QS6210: The type of the given argument does not match the expected type. Got an argument of type Int, expecting one of type Unit instead.
C:/snippet:30b12e50-b7e8-41a9-aaad-a933ace9be5f(2,12): error QS6301: The type Result of the given expression is not compatible with the expected return type Int[].


In [ ]:
%simulate Qrng

In [28]:
open Microsoft.Quantum.Extensions.Diagnostics;
operation QrngWithInfo() : Result {

    using (qubit = Qubit()) {
    
        Message("Here is what the simulator uses to record a qubit in the 0 state:");
        DumpRegister((), [qubit]);

        Message(" ");
        
        H(qubit);
        
        Message("Here is what the simulator uses to record a superposition state:");
        DumpRegister((), [qubit]);
        
        return MResetZ(qubit);
    }
}

QrngWithInfo

In [34]:
%simulate QrngWithInfo

Here is what the simulator uses to record a qubit in the 0 state:
# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0
 
Here is what the simulator uses to record a superposition state:
# wave function for qubits with ids (least to most significant): 0
0:	0.707106781186548	0
1:	0.707106781186548	0


Zero

In [35]:
operation Qrng(nDigits : Int) : Int[] {

    // Start with an array for the results:
    mutable bitList = new Int[nDigits];
    
    // Loop for nDigits, preparing an entangled state and then measurning it
    for (i in 0..nDigits - 1) {
        using (qubit = Qubit()) {
            H(qubit);
            set bitList[i] = ResultAsInt([MResetZ(qubit)]);
        }  
    }

    return bitList;
}

Qrng

In [36]:
operation Qrng10bit(): Int[] {
    return Qrng(10);
}

Qrng10bit

In [37]:
%simulate Qrng10bit

0, 0, 0, 1, 1, 0, 1, 0, 1, 1

# Deutsch–Jozsa

In [ ]:
operation ZeroOracle(control : Qubit, target : Qubit) : Unit {
}

operation OneOracle(control : Qubit, target : Qubit) : Unit {
    X(target);
}

operation IdOracle(control : Qubit, target : Qubit) : Unit {
    CNOT(control, target);
}

operation NotOracle(control : Qubit, target : Qubit) : Unit {
    X(control);
    CNOT(control, target);
    X(control);
}

In [ ]:
operation IsOracleBalanced( oracle : ((Qubit, Qubit) => Unit)) : Bool {
    mutable result = Zero;
    using ((control, target) = (Qubit(), Qubit())) {
        H(control);
        X(target);
        H(target);
        oracle(control, target);
        H(target); 
        X(target);
        set result = MResetX(control);
    }
    return result == One;
}

In [ ]:
operation IsOZeroOracleBalanced() : Bool{
    return IsOracleBalanced(ZeroOracle);
}

operation IsOneOracleBalanced() : Bool{
    return IsOracleBalanced(OneOracle);
}

operation IsIdOracleBalanced() : Bool{
    return IsOracleBalanced(IdOracle);
}

operation IsNotOracleBalanced() : Bool{
    return IsOracleBalanced(NotOracle);
}

In [ ]:
%simulate IsOneOracleBalanced

In [ ]:
%simulate IsOneOracleBalanced

In [ ]:
%simulate IsIdOracleBalanced

In [ ]:
%simulate IsNotOracleBalanced